In [1]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

# S3 prefix
bucket = "humana-data"
prefix = "humana-Linearclass-pipeline-condition-credit"

In [2]:
train_input = 's3://{}/rawdata/original_raw_files/train'.format(bucket)
train_input

's3://humana-data/rawdata/original_raw_files/train'

In [8]:
valid_input = 's3://{}/rawdata/original_raw_files/valid'.format(bucket)
valid_input

's3://humana-data/rawdata/original_raw_files/valid'

In [10]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "1.0-1"
script_dir = "humana_script"
script_path = "humana_transportation_featurizer.py"

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    source_dir=script_dir,
    role=role,
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.c4.xlarge",
    sagemaker_session=sagemaker_session,
)

In [11]:
sklearn_preprocessor.fit({"train": train_input})

2022-11-23 02:00:07 Starting - Starting the training job...
2022-11-23 02:00:38 Starting - Preparing the instances for trainingProfilerReport-1669168807: InProgress
.........
2022-11-23 02:01:56 Downloading - Downloading input data...
2022-11-23 02:02:36 Training - Downloading the training image...
2022-11-23 02:03:06 Training - Training image download completed. Training in progress..2022-11-23 02:03:09,604 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-11-23 02:03:09,607 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-11-23 02:03:09,616 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-11-23 02:03:10,151 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-11-23 02:03:10,163 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-11-23 02:03:10,174 sagemaker-training-toolkit INFO     No GPUs detected (no

In [12]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, instance_type="ml.c4.xlarge", assemble_with="Line", accept="text/csv"
)

In [6]:
# Preprocess training input
transformer.transform(train_input, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path

................................2022-11-23 01:41:45,094 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2022-11-23 01:41:45,096 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2022-11-23 01:41:45,097 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      

Traceback (most recent call last):
  File "/miniconda3/lib/python3.8/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/miniconda3/lib/python3.8/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/miniconda3/lib/python3.8/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/miniconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/miniconda3/lib/python3.8/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/miniconda3/lib/python3.8/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_containers/_transformer.py", line 199, in transform
    result = self._transform_fn(
  File "/miniconda3/lib/py

UnexpectedStatusException: Error for Transform job sagemaker-scikit-learn-2022-11-23-01-36-32-824: Failed. Reason: AlgorithmError: See job logs for more information

In [13]:
# Preprocess training input
transformer.transform(valid_input, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_valid = transformer.output_path

...............................2022-11-23 02:09:21,273 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2022-11-23 02:09:21,275 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2022-11-23 02:09:21,276 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      r

2022-11-23T02:09:29.466:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
2022-11-23T02:09:30.444:[sagemaker logs]: humana-data/rawdata/original_raw_files/valid/dependent.csv: Bad HTTP status received from algorithm: 500
2022-11-23T02:09:30.444:[sagemaker logs]: humana-data/rawdata/original_raw_files/valid/dependent.csv: 
2022-11-23T02:09:30.444:[sagemaker logs]: humana-data/rawdata/original_raw_files/valid/dependent.csv: Message:
2022-11-23T02:09:30.445:[sagemaker logs]: humana-data/rawdata/original_raw_files/valid/dependent.csv: <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
2022-11-23T02:09:30.445:[sagemaker logs]: humana-data/rawdata/original_raw_files/valid/dependent.csv: <title>500 Internal Server Error</title>
2022-11-23T02:09:30.445:[sagemaker logs]: humana-data/rawdata/original_raw_files/valid/dependent.csv: <h1>Internal Server Error</h1>
2022-11-23T02:09:30.445:[sagemaker logs]: humana-data/rawdata/original_raw_files/valid/depe

UnexpectedStatusException: Error for Transform job sagemaker-scikit-learn-2022-11-23-02-04-19-992: Failed. Reason: AlgorithmError: See job logs for more information

In [10]:
import boto3
from sagemaker.image_uris import retrieve

ll_image = retrieve("linear-learner", boto3.Session().region_name)

In [11]:
s3_ll_output_key_prefix = "ll_training_output"
s3_ll_output_location = "s3://{}/{}/{}/{}".format(
    bucket, prefix, s3_ll_output_key_prefix, "ll_model"
)

ll_estimator = sagemaker.estimator.Estimator(
    ll_image,
    role,
    instance_count=1,
    instance_type="ml.m4.2xlarge",
    volume_size=20,
    max_run=3600,
    input_mode="File",
    output_path=s3_ll_output_location,
    sagemaker_session=sagemaker_session,
)

ll_estimator.set_hyperparameters(feature_dim=10, predictor_type="regressor", mini_batch_size=32)

ll_train_data = sagemaker.inputs.TrainingInput(
    preprocessed_train,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)

data_channels = {"train": ll_train_data}
ll_estimator.fit(inputs=data_channels, logs=True)

NameError: name 'preprocessed_train' is not defined